<a href="https://colab.research.google.com/github/Matthev00/WMM/blob/main/LAB7/face_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## WMM - LAB7
Mateusz Ostaszewski 325203 14.05.2024 Grupa 102

# Analiza obrazu - detekcja twarzy

## Inicjalizacja
import bibliotek, pobranie do lokalnego środowiska wykonawczego przykładowego obrazu testowego i modeli dla detektorów twarzy

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

import dlib

# biblioteka insightface wymaga zainstalowania
!pip install insightface
!pip install onnxruntime

import insightface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 64.9 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp310-cp310-linux_x86_64.whl size=1054142 sha256=10af802e975977611540c22c9fd2a66e38c0feee7131dd70c45c005899d690ed
  Stored in directory: /root/.cache/pip/wheels/e3/d0/80/e3773fb8b6d1cca87ea1d33d9b1f20a223a6493c896da249b5
Successfully built insightface
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.4 MB/s eta 0:00:00


In [ ]:
# obraz testowy
# https://drive.google.com/file/d/1dKxVJO0n5O6VuiHDdvrS36touzAeCgbw/view?usp=sharing
!gdown 1dKxVJO0n5O6VuiHDdvrS36touzAeCgbw

Downloading...
From: https://drive.google.com/uc?id=1dKxVJO0n5O6VuiHDdvrS36touzAeCgbw
To: /content/2_Demonstration_Demonstration_Or_Protest_2_58.jpg
100% 173k/173k [00:00<00:00, 45.3MB/s]


In [ ]:
# model detektora Haar
# https://drive.google.com/file/d/1k2J4-4yIXrFFHZUhFv4pCRqcl-wB2Ve9/view?usp=sharing
!gdown 1k2J4-4yIXrFFHZUhFv4pCRqcl-wB2Ve9
!unzip haarcascade_frontalface_default.zip

Downloading...
From: https://drive.google.com/uc?id=1k2J4-4yIXrFFHZUhFv4pCRqcl-wB2Ve9
To: /content/haarcascade_frontalface_default.zip
100% 140k/140k [00:00<00:00, 72.6MB/s]
Archive:  haarcascade_frontalface_default.zip
replace haarcascade_frontalface_default.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# model detektora MMOD (sieć neuronowa z biblioteki dlib)
# https://drive.google.com/file/d/1gwFX_zny4A6DVkQ2gV2FlKY39qh1iDYj/view?usp=sharing
!gdown 1gwFX_zny4A6DVkQ2gV2FlKY39qh1iDYj

In [ ]:
# modele detektora twarzy (siec neuronowa) z biblioteki insightface
!gdown 1avyNxM0XoYh8YfxhhZoCvvI0lE7qT8Gy
!unzip insightface.zip

## Metryki


In [ ]:
def calculate_recall(true_positives, false_negatives):
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0
    return recall

def calculate_precision(true_positives, false_positives):
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 0
    return precision

def calculate_f1_score(true_positives, false_positives, true_negatives, false_negatives):
    precision = calculate_precision(true_positives, false_positives)
    recall = calculate_recall(true_positives, false_negatives)
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    return f1_score

def calculate_accuracy(true_positives, false_positives, true_negatives, false_negatives):
    total_predictions = true_positives + false_positives + true_negatives + false_negatives
    accuracy = (true_positives + true_negatives) / total_predictions if total_predictions != 0 else 0
    return accuracy

## Obraz testowy

In [ ]:
img = cv2.imread("2_Demonstration_Demonstration_Or_Protest_2_58.jpg")
# przygotowanie obrazów: monochromatycznego i RGB (cv2.imread() zwraca obraz w formacie BGR - inna kolejność składowych)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

cv2_imshow(img)

## Kaskada Haara

Opracowano na podstawie: https://www.pyimagesearch.com/2021/04/05/opencv-face-detection-with-haar-cascades/

In [ ]:
# utworzenie i inicjalizacja detektora
haar_detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [ ]:
# wywołanie detektora dla określonego obrazu (img_gray)
# wynikiem jest lista prostokątów w formacie [x, y, width, height]
haar_results = haar_detector.detectMultiScale(img_gray, scaleFactor=1.05,
                                         minNeighbors=5, minSize=(30, 30),
																				 flags=cv2.CASCADE_SCALE_IMAGE)
print(len(haar_results))

# narysowanie wyników na kopii obrazu i wyświetlenie
img_haar = img.copy()
for (x, y, w, h) in haar_results:
  cv2.rectangle(img_haar, (x, y), (x+w, y+h), (0, 255, 0), 2)
cv2_imshow(img_haar)

In [ ]:
print(calculate_accuracy(20, 3, 0, 81))
print(calculate_f1_score(20, 3, 0, 81))

### Haar Results

| Ground Truth | TP | FN | FP | Accuracy | F1 |
|------------  |----|----|----|----------|----|
|     101      | 20 | 81 |  3 | 0,19     | 0,32|


## Histogram zorientowanych gradientów (HOG) z maszyną wektorów nośnych (SVM)

Opracowano na podstawie: https://www.pyimagesearch.com/2021/04/19/face-detection-with-dlib-hog-and-cnn/

In [ ]:
# utworzenie i inicjalizacja detektora
hog_svm_detector = dlib.get_frontal_face_detector()

In [ ]:
# wywołanie detektora dla określonego obrazu (img_rgb)
# wynikiem jest lista obiektów rectangle, zawierających współrzędne lewego górnego i prawego dolnego narożnika prostokąta
hog_svm_results = hog_svm_detector(img_rgb, 2)
print(len(hog_svm_results))

# narysowanie wyników na kopii obrazu i wyświetlenie
img_hog_svm = img.copy()
for rect in hog_svm_results:
	cv2.rectangle(img_hog_svm, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
cv2_imshow(img_hog_svm)

In [ ]:
print(calculate_accuracy(29, 0, 0, 72))
print(calculate_f1_score(29, 0, 0, 72))

### HOG SVM Results

| Ground Truth | TP | FN | FP | Accuracy | F1 |
|------------  |----|----|----|----------|----|
|     101      | 29 | 72 |  0 | 0,29     | 0,45|

Przy wiekszym parametrze k-krotnego sklaowania mamy lepszy efekt ale zajmuje to zdecydowanie wiecej czasu optymalna okazuje sie wartość 2.

## Splotowa sieć neuronowa (CNN) z biblioteki dlib

Opracowano na podstawie: https://www.pyimagesearch.com/2021/04/19/face-detection-with-dlib-hog-and-cnn/

In [ ]:
# utworzenie i inicjalizacja detektora
cnn1_detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')

In [ ]:
# wywołanie detektora dla określonego obrazu (img_rgb)
# wynikiem jest lista obiektów mmod_rectangle, zawierających m.in. pole rect ze współrzędnymi lewego górnego i prawego dolnego narożnika prostokąta
cnn1_results = cnn1_detector(img_rgb, 3)
print(len(cnn1_results))

# narysowanie wyników na kopii obrazu i wyświetlenie
img_cnn1 = img.copy()
for res in cnn1_results:
	rect = res.rect
	cv2.rectangle(img_cnn1, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
cv2_imshow(img_cnn1)

In [ ]:
print(calculate_accuracy(46, 0, 0, 55))
print(calculate_f1_score(46, 0, 0, 55))

### CNN Results

| Ground Truth | TP | FN | FP | Accuracy | F1 |
|------------  |----|----|----|----------|----|
|     101      | 46 | 55 |  0 | 0,46     | 0,63|

Przy wiekszym parametrze k-krotnego sklaowania mamy lepszy efekt ale zajmuje to wiecej czasu(ale mniej niż w HOG SVM). Optymalna okazuje sie wartość 3.

## InsightFace - inna sieć neuronowa

Opracowano na podstawie: https://github.com/deepinsight/insightface/tree/master/python-package

In [ ]:
# utworzenie i inicjalizacja detektora
model_name = 'buffalo_s'  # model: small (_s), medium (_m) lub large (_l)
insf_detector = insightface.app.FaceAnalysis(name=model_name, root='insightface',
                                             allowed_modules=['detection'], providers=['CPUExecutionProvider'])
insf_detector.prepare(ctx_id=0, det_size=(1024, 1024))

In [ ]:
# wywołanie detektora dla określonego obrazu (img_rgb)
# wynikiem jest lista obiektów, zawierających m.in. pole bbox ze współrzędnymi lewego górnego i prawego dolnego narożnika prostokąta
insf_results = insf_detector.get(img)
print(len(insf_results))

# narysowanie wyników na kopii obrazu i wyświetlenie
img_insf = img.copy()
for res in insf_results:
  # współrzędne prostokątów sa zapisane jako liczby rzeczywiste - konwersja do liczb całkowitych
  rect = res.bbox.round().astype(int)
  cv2.rectangle(img_insf, (rect[0], rect[1]), (rect[2], rect[3]), (0, 255, 0), 2)
cv2_imshow(img_insf)

In [ ]:
print(calculate_accuracy(56, 0, 0, 45))
print(calculate_f1_score(56, 0, 0, 45))

### InsightFace Results

| Ground Truth | TP | FN | FP | Accuracy | F1 |
|------------  |----|----|----|----------|----|
|     101      | 56 | 45 |  0 |0.55|0.71|

# WYNIKI ZAD1
|             | Ground Truth | TP | FN | FP | Accuracy | F1 |
|-------------|------------  |----|----|----|----------|----|
| Haar        |     101      | 20 | 81 |  3 | 0,19     |0,32|
| HOG SVM     |     101      | 29 | 72 |  0 | 0,29     |0,45|
| CNN         |     101      | 46 | 55 |  0 | 0,46     |0,63|
| InsightFace |     101      | 56 | 45 |  0 | 0.55     |0.71|

Najskuteczniejsze okazują sie sieci neuronowe w szczegolności InsightFace, CNN(3) rownież ma potencjał. Parametr k-krotnej walidacji powyżej 3 powoduje bardzo wolne przetwarzanie obrazu, ktore jest niepraktyczne w większości zastosowań.

#ZAD2

## Wczytanie zdjęć

In [ ]:
img1 = cv2.imread("/content/12_Group_Group_12_Group_Group_12_478.jpg")
img_gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img_rgb1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
num_img1_faces =  5

img2 = cv2.imread("/content/20_Family_Group_Family_Group_20_447.jpg")
img_gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img_rgb2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
num_img2_faces = 6

img3 = cv2.imread("/content/8_Election_Campain_Election_Campaign_8_236.jpg")
img_gray3 = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)
img_rgb3 = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
num_img3_faces = 5


# cv2_imshow(img1)
# cv2_imshow(img2)
cv2_imshow(img3)

## IMG 1


In [ ]:
haar_results = haar_detector.detectMultiScale(img_gray1, scaleFactor=1.05,
                                        minNeighbors=5, minSize=(30, 30),
                                        flags=cv2.CASCADE_SCALE_IMAGE)
img_haar1 = img1.copy()
for (x, y, w, h) in haar_results:
  cv2.rectangle(img_haar1, (x, y), (x+w, y+h), (0, 255, 0), 2)
cv2_imshow(img_haar1)
print(len(haar_results))
print(calculate_accuracy(2, 0, 0, num_img1_faces - len(haar_results)))
print(calculate_f1_score(2, 0, 0, num_img1_faces - len(haar_results)))

In [ ]:
hog_svm_results = hog_svm_detector(img_rgb1, 2)

# narysowanie wyników na kopii obrazu i wyświetlenie
img_hog_svm = img1.copy()
for rect in hog_svm_results:
  cv2.rectangle(img_hog_svm, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
cv2_imshow(img_hog_svm)
faces = len(hog_svm_results)
print(faces)
print(calculate_accuracy(faces, 0, 0, num_img1_faces - faces))
print(calculate_f1_score(faces, 0, 0, num_img1_faces - faces))

In [ ]:
cnn1_results = cnn1_detector(img_rgb1, 3)

img_cnn1 = img1.copy()
for res in cnn1_results:
	rect = res.rect
	cv2.rectangle(img_cnn1, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
cv2_imshow(img_cnn1)
faces = len(cnn1_results)
print(faces)
print(calculate_accuracy(faces, 0, 0, num_img1_faces - faces))
print(calculate_f1_score(faces, 0, 0, num_img1_faces - faces))

In [ ]:
insf_results = insf_detector.get(img1)

img_insf = img1.copy()
for res in insf_results:
  # współrzędne prostokątów sa zapisane jako liczby rzeczywiste - konwersja do liczb całkowitych
  rect = res.bbox.round().astype(int)
  cv2.rectangle(img_insf, (rect[0], rect[1]), (rect[2], rect[3]), (0, 255, 0), 2)
cv2_imshow(img_insf)
faces = len(insf_results)
print(faces)
print(calculate_accuracy(faces, 0, 0, num_img1_faces - faces))
print(calculate_f1_score(faces, 0, 0, num_img1_faces - faces))

### Wnioski
|             | Ground Truth | TP | FN | FP | Accuracy | F1 |
|-------------|------------  |----|----|----|----------|----|
| Haar        |     5        | 2  | 3  |  0 | 0,4      |0,57|
| HOG SVM     |     5        | 3  | 2  |  0 | 0,6      |0,75|
| CNN         |     5        | 4  | 1  |  0 | 0,8      |0,89|
| InsightFace |     5        | 3  | 2  |  0 | 0,6      |0,75|

Najlepsza okazała się sieć CNN. Zaskakujące może być to że HOG SVM był równie dobry co InsightFace.

## IMG 2

In [ ]:
haar_results = haar_detector.detectMultiScale(img_gray2, scaleFactor=1.05,
                                        minNeighbors=5, minSize=(30, 30),
                                        flags=cv2.CASCADE_SCALE_IMAGE)
img_haar1 = img2.copy()
for (x, y, w, h) in haar_results:
  cv2.rectangle(img_haar1, (x, y), (x+w, y+h), (0, 255, 0), 2)
cv2_imshow(img_haar1)
print(1)
print(calculate_accuracy(1, 3, 0, 5))
print(calculate_f1_score(1, 3, 0, 5))

In [ ]:
hog_svm_results = hog_svm_detector(img_rgb2, 2)

# narysowanie wyników na kopii obrazu i wyświetlenie
img_hog_svm = img2.copy()
for rect in hog_svm_results:
  cv2.rectangle(img_hog_svm, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
cv2_imshow(img_hog_svm)
faces = len(hog_svm_results)
print(faces)
print(calculate_accuracy(faces, 0, 0, num_img2_faces - faces))
print(calculate_f1_score(faces, 0, 0, num_img2_faces - faces))

In [ ]:
cnn1_results = cnn1_detector(img_rgb2, 3)

img_cnn1 = img2.copy()
for res in cnn1_results:
	rect = res.rect
	cv2.rectangle(img_cnn1, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
cv2_imshow(img_cnn1)
faces = len(cnn1_results)
print(faces)
print(calculate_accuracy(3, 1, 0, 3))
print(calculate_f1_score(3, 1, 0, 3))

In [ ]:
insf_results = insf_detector.get(img2)

img_insf = img2.copy()
for res in insf_results:
  # współrzędne prostokątów sa zapisane jako liczby rzeczywiste - konwersja do liczb całkowitych
  rect = res.bbox.round().astype(int)
  cv2.rectangle(img_insf, (rect[0], rect[1]), (rect[2], rect[3]), (0, 255, 0), 2)
cv2_imshow(img_insf)
faces = len(insf_results)
print(faces)
print(calculate_accuracy(faces, 0, 0, num_img2_faces - faces))
print(calculate_f1_score(faces, 0, 0, num_img2_faces - faces))

### Wnioski
|             | Ground Truth | TP | FN | FP | Accuracy | F1 |
|-------------|------------  |----|----|----|----------|----|
| Haar        |     6        | 1  | 5  |  3 | 0,1      |0,2 |
| HOG SVM     |     6        | 1  | 5  |  0 | 0,17     |0,29|
| CNN         |     6        | 4  | 2  |  1 | 0,43     |0,6 |
| InsightFace |     6        | 3  | 3  |  0 | 0,5      |0,66|

Choć obraz wydaje się prosty do zadania detekcji wyniki mówią co innego. Najlepiej jak zwykle sprawdzily się sieci neuronowe w szczególności InsightFace. Widzimy też dużą skłonnośc algorytmu Haar do FP.

## IMG 3

In [ ]:
haar_results = haar_detector.detectMultiScale(img_gray3, scaleFactor=1.05,
                                        minNeighbors=5, minSize=(30, 30),
                                        flags=cv2.CASCADE_SCALE_IMAGE)
img_haar1 = img3.copy()
for (x, y, w, h) in haar_results:
  cv2.rectangle(img_haar1, (x, y), (x+w, y+h), (0, 255, 0), 2)
cv2_imshow(img_haar1)
faces = 2
print(faces)
print(calculate_accuracy(faces, 2, 0, num_img3_faces - faces))
print(calculate_f1_score(faces, 2, 0, num_img3_faces - faces))

In [ ]:
hog_svm_results = hog_svm_detector(img_rgb3, 2)

# narysowanie wyników na kopii obrazu i wyświetlenie
img_hog_svm = img3.copy()
for rect in hog_svm_results:
  cv2.rectangle(img_hog_svm, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
cv2_imshow(img_hog_svm)
faces = len(hog_svm_results)
print(faces)
print(calculate_accuracy(faces, 0, 0, num_img3_faces - faces))
print(calculate_f1_score(faces, 0, 0, num_img3_faces - faces))

In [ ]:
cnn1_results = cnn1_detector(img_rgb3, 3)

img_cnn1 = img3.copy()
for res in cnn1_results:
	rect = res.rect
	cv2.rectangle(img_cnn1, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
cv2_imshow(img_cnn1)
faces = len(cnn1_results)
print(faces)
print(calculate_accuracy(faces, 0, 0, num_img3_faces - faces))
print(calculate_f1_score(faces, 0, 0, num_img3_faces - faces))

In [ ]:
insf_results = insf_detector.get(img3)

img_insf = img3.copy()
for res in insf_results:
  # współrzędne prostokątów sa zapisane jako liczby rzeczywiste - konwersja do liczb całkowitych
  rect = res.bbox.round().astype(int)
  cv2.rectangle(img_insf, (rect[0], rect[1]), (rect[2], rect[3]), (0, 255, 0), 2)
cv2_imshow(img_insf)
faces = len(insf_results)
print(faces)
print(calculate_accuracy(faces, 0, 0, num_img3_faces - faces))
print(calculate_f1_score(faces, 0, 0, num_img3_faces - faces))

### Wnioski
|             | Ground Truth | TP | FN | FP | Accuracy | F1 |
|-------------|------------  |----|----|----|----------|----|
| Haar        |     5        | 2  | 3  |  2 | 0,29     |0,44|
| HOG SVM     |     5        | 1  | 4  |  0 | 0,2      |0,33|
| CNN         |     5        | 4  | 1  |  0 | 0,8      |0,89|
| InsightFace |     5        | 5  | 0  |  0 | 1        |1   |

Detektor InsightFace w przypadku tego zdjecia okazal się bezbłędny. CCN też sobie dobrze poradził. HOG SVM i Haar okazały się na tym zdjęciu zdecydowanie gorsze.

# ZAD3

In [ ]:
total_acc_haar = calculate_accuracy(25, 8, 0, 92)
total_acc_hog = calculate_accuracy(34, 0, 0, 83)
total_acc_cnn = calculate_accuracy(58, 1, 0, 59)
total_acc_if = calculate_accuracy(67, 0, 0, 50)
print(total_acc_haar)
print(total_acc_hog)
print(total_acc_cnn)
print(total_acc_if)

In [ ]:
total_f1_haar = calculate_f1_score(25, 8, 0, 92)
total_f1_hog = calculate_f1_score(34, 0, 0, 83)
total_f1_cnn = calculate_f1_score(58, 1, 0, 59)
total_f1_if = calculate_f1_score(67, 0, 0, 50)
print(total_f1_haar)
print(total_f1_hog)
print(total_f1_cnn)
print(total_f1_if)

## Wnioski

|             | Ground Truth | TP | FN | FP | Accuracy | F1 |
|-------------|------------  |----|----|----|----------|----|
| Haar        |     117      | 25 | 92 |  8 | 0,2      |0,33|
| HOG SVM     |     117      | 34 | 83 |  0 | 0,29     |0,45|
| CNN         |     117      | 58 | 59 |  1 | 0,49     |0,66|
| InsightFace |     117      | 67 | 50 |  0 | 0,57     |0,73|

Ze względu na dużą dysproporcję ilości twarzy na pierwszym zdjęciu w porównaniu do pozostałych, rezultaty są bardzo podobne do tych z ZAD 1. Podsumowując skuteczność modeli, można zauważyć, że InsightFace charakteryzuje się najlepszymi wynikami i wyraźnie wyróżnia się w detekcji "trudnych" zdjęć. Model CNN również osiąga dobre wyniki, szczególnie przy prostszych zdjęciach, gdzie może nawet przewyższać InsightFace.

Z kolei modele Haar i HOG SVM mają zdecydowanie gorsze rezultaty. Algorytm Haar wykazuje największą tendencję do błędnego rozpoznawania części obrazu jako twarze (FP), co obniża jego skuteczność. HOG SVM również nie radzi sobie tak dobrze jak CNN czy InsightFace, zwłaszcza w przypadku zdjęć o wyższym stopniu trudności.

Wyniki te wskazują, że dla zadań detekcji twarzy na złożonych obrazach InsightFace jest najbardziej efektywnym wyborem, a CNN może być dobrym rozwiązaniem w prostszych scenariuszach. Haar i HOG SVM natomiast mogą nie być wystarczająco precyzyjne w bardziej wymagających zastosowaniach.

#ZAD4


In [ ]:
from timeit import default_timer as timer

In [ ]:
iters = 5
haar_times = []
hog_svm_times_0 = []
hog_svm_times_1 = []
hog_svm_times_2 = []
cnn_times_0 = []
cnn_times_1 = []
cnn_times_3 = []
insightFace_times =[]
for i in range(iters):
  start = timer()
  haar_detector.detectMultiScale(img_gray, scaleFactor=1.05,
                                minNeighbors=5, minSize=(30, 30),
                                flags=cv2.CASCADE_SCALE_IMAGE)
  haar_times.append(timer()-start)

  start = timer()
  hog_svm_detector(img_rgb, 0)
  hog_svm_times_0.append(timer()-start)

  start = timer()
  hog_svm_detector(img_rgb, 1)
  hog_svm_times_1.append(timer()-start)

  start = timer()
  hog_svm_detector(img_rgb, 2)
  hog_svm_times_2.append(timer()-start)

  start = timer()
  cnn1_detector(img_rgb, 0)
  cnn_times_0.append(timer() - start)

  start = timer()
  cnn1_detector(img_rgb, 1)
  cnn_times_1.append(timer() - start)

  start = timer()
  cnn1_detector(img_rgb, 3)
  cnn_times_3.append(timer() - start)

  start = timer()
  insf_detector.get(img)
  insightFace_times.append(timer() - start)




In [ ]:
import matplotlib.pyplot as plt

def plot_detection_times(haar_times, hog_svm_times_0, hog_svm_times_1, hog_svm_times_2, cnn_times_0, cnn_times_1, cnn_times_3, insightFace_times):
    iters = len(haar_times)
    x = range(iters)

    plt.figure(figsize=(14, 8))

    # Plotting Haar times
    plt.plot(x, haar_times, label='Haar')

    # Plotting HOG + SVM times
    plt.plot(x, hog_svm_times_0, label='HOG + SVM (0)')
    plt.plot(x, hog_svm_times_1, label='HOG + SVM (1)')
    plt.plot(x, hog_svm_times_2, label='HOG + SVM (2)')

    # Plotting CNN times
    plt.plot(x, cnn_times_0, label='CNN (0)')
    plt.plot(x, cnn_times_1, label='CNN (1)')
    plt.plot(x, cnn_times_3, label='CNN (3)')

    # Plotting InsightFace times
    plt.plot(x, insightFace_times, label='InsightFace')

    plt.xlabel('Iteration')
    plt.ylabel('Detection Time (seconds)')
    plt.title('Detection Times for Different Detectors')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
import pandas as pd

def print_detection_times_table(haar_times, hog_svm_times_0, hog_svm_times_1, hog_svm_times_2, cnn_times_0, cnn_times_1, cnn_times_3, insightFace_times):
    data = {
        'Iteration': list(range(1, len(haar_times) + 1)),
        'Haar': haar_times,
        'HOG + SVM (0)': hog_svm_times_0,
        'HOG + SVM (1)': hog_svm_times_1,
        'HOG + SVM (2)': hog_svm_times_2,
        'CNN (0)': cnn_times_0,
        'CNN (1)': cnn_times_1,
        'CNN (3)': cnn_times_3,
        'InsightFace': insightFace_times
    }

    df = pd.DataFrame(data)

    # Calculate means
    mean_row = df.mean(numeric_only=True).to_frame().T
    mean_row['Iteration'] = 'Mean'

    # Append the mean row to the dataframe
    df = pd.concat([df, mean_row], ignore_index=True)

    print(df)

In [ ]:
plot_detection_times(haar_times, hog_svm_times_0, hog_svm_times_1, hog_svm_times_2, cnn_times_0, cnn_times_1, cnn_times_3, insightFace_times)

In [ ]:
print_detection_times_table(haar_times, hog_svm_times_0, hog_svm_times_1, hog_svm_times_2, cnn_times_0, cnn_times_1, cnn_times_3, insightFace_times)


##Wnioski
Na wygenerowanym wykresie można zauważyć, że czasy działania większości detektorów są stabilne, z wyjątkiem algorytmu Haar, który wykazuje pewne wahania, oraz detektora HOG SVM z parametrem skalowania równym 2. Analiza średnich czasów zamieszczonych w tabeli oraz wykresu wskazuje, że parametr skalowania ma istotny wpływ na czas działania zarówno w przypadku detektorów CNN, jak i HOG SVM – im większy parametr skalowania, tym dłuższy czas działania.

Przy porównaniu czasów działania i skuteczności różnych detektorów, model InsightFace wypada bardzo korzystnie. Charakteryzuje się on krótkim czasem wykrywania, co w połączeniu z wysoką skutecznością czyni go bardzo efektywnym narzędziem w zadaniach detekcji.

Użyte środowisko to: Google Compute Engine języka Python 3 z układem (GPU) Nvidia T4